In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import Models.Corn_USA_Yield as cy
import Utilities.GLOBAL as GV
import Utilities.Weather as uw
import Utilities.SnD as us
from datetime import datetime as dt

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
scope = cy.Define_Scope()
scope.keys()

dict_keys(['geo_df', 'geo_input_file', 'geo_output_column', 'w_vars', 'years'])

In [4]:
raw_data = cy.Get_Data(scope)
raw_data.keys()

dict_keys(['yield', 'planting_progress', 'silking_progress', 'w_df_all', 'locations', 'weights', 'w_w_df_all'])

In [6]:
processed_data = cy.Process_Data(scope,raw_data)
instructions = cy.Build_Model_DF_Instructions('weighted',GV.WD_HIST, prec_units='mm', temp_units='C')
df = cy.Build_Model_DF(scope, raw_data, processed_data, instructions)
model = cy.Fit_Model(df,'Yield',GV.CUR_YEAR)
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Yield   R-squared:                       0.976
Model:                            OLS   Adj. R-squared:                  0.968
Method:                 Least Squares   F-statistic:                     122.8
Date:                Fri, 17 Jun 2022   Prob (F-statistic):           1.53e-19
Time:                        14:51:22   Log-Likelihood:                -99.986
No. Observations:                  37   AIC:                             220.0
Df Residuals:                      27   BIC:                             236.1
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                   -3

In [ ]:
date_start = dt(2022,6,10)
date_end = raw_data['w_w_df_all'][GV.WD_H_GFS].index[-1]   

days = list(pd.date_range(date_start, date_end))

df.loc[[1985,1987]]

df_pred = pd.DataFrame(columns= df.columns,index=days)
df_pred

In [ ]:
uw.open_w_sel_file()